**Esercizio**

In [7]:
import os
import dotenv
import sqlalchemy
import pandas as pd
dotenv.load_dotenv(override=True, dotenv_path= r"C:\Users\patri\OneDrive\Documents\Epicode Data Analyst\Modulo 3 - Python\W10D1\.env") 

username = os.getenv("username") 
password = os.getenv("password") 
host = os.getenv("host") 
dbname = os.getenv("dbname") 
connection_string = "mysql+pymysql://"+username+":"+password+"@"+host+"/"+dbname

db_engine = sqlalchemy.create_engine(connection_string)

In [14]:
# Dal database AdventureWorksDW importiamo le tabelle dimemployee e dimemployeesalesterritory come DataFrame 
query = "SELECT * FROM dimemployee" 
employee = pd.read_sql(query, db_engine) 
query = "SELECT * FROM dimemployeesalesterritory" 
sales_terr = pd.read_sql(query, db_engine) 

In [25]:
# Effettuiamo un join tra i due DataFrame usando le colonne EmployeeKey 
employee_terr = pd.merge(left=employee, right=sales_terr, how="left", on="EmployeeKey")

In [29]:
# Controlliamo la dimensione del DataFrame risultante: è quella attesa? 
employee.shape

(296, 28)

In [31]:
sales_terr.shape

(39, 2)

In [33]:
employee_terr.shape

(317, 29)

In [78]:
# Importiamo ora la tabella  ed effettuiamo un join tra questa e il DataFrame risultante della join precedente, usando le colonne SalesTerritoryKey
query = "SELECT * FROM dimsalesterritory" 
terr = pd.read_sql(query, db_engine)
emp_sal_terr = pd.merge(left=employee_terr, right = terr, how="left", on="SalesTerritoryKey")

In [80]:
emp_sal_terr.head(2)

,EmployeeKey,ParentEmployeeKey,EmployeeNationalIDAlternateKey,ParentEmployeeNationalIDAlternateKey,FirstName,LastName,MiddleName,NameStyle,Title,HireDate,...,SalesPersonFlag,DepartmentName,EmployeePhoto,Position,SalesTerritoryKey,SalesTerritoryAlternateKey,SalesTerritoryRegion,SalesTerritoryCountry,SalesTerritoryGroup,SalesTerritoryImage
0,1,18.0,14417807,446466105.0,Guy,Gilbert,R,0,Production Technician - WC60,2006-01-28,...,0,Production,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,None,NaN,NaN,NaN,NaN,NaN,NaN
1,2,7.0,253022876,24756624.0,Kevin,Brown,F,0,Marketing Assistant,2006-08-26,...,0,Marketing,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,None,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
emp_sal_terr.columns

Index(['EmployeeKey', 'ParentEmployeeKey', 'EmployeeNationalIDAlternateKey',
       'ParentEmployeeNationalIDAlternateKey', 'FirstName', 'LastName',
       'MiddleName', 'NameStyle', 'Title', 'HireDate', 'BirthDate', 'LoginID',
       'EmailAddress', 'Phone', 'MaritalStatus', 'EmergencyContactName',
       'EmergencyContactPhone', 'SalariedFlag', 'Gender', 'PayFrequency',
       'BaseRate', 'VacationHours', 'SickLeaveHours', 'CurrentFlag',
       'SalesPersonFlag', 'DepartmentName', 'EmployeePhoto', 'Position',
       'SalesTerritoryKey', 'SalesTerritoryAlternateKey',
       'SalesTerritoryRegion', 'SalesTerritoryCountry', 'SalesTerritoryGroup',
       'SalesTerritoryImage'],
      dtype='object')

In [101]:
# Su questo DataFrame contare quanti dipendenti ci sono per ogni paese (country) e per ogni regione (region) 
emp_sal_terr.groupby(["SalesTerritoryCountry","SalesTerritoryRegion"]).EmployeeKey.count()

SalesTerritoryCountry  SalesTerritoryRegion
Australia              Australia               3
Canada                 Canada                  4
France                 France                  4
Germany                Germany                 3
United Kingdom         United Kingdom          3
United States          Central                 5
                       Northeast               3
                       Northwest               5
                       Southeast               4
                       Southwest               5
Name: EmployeeKey, dtype: int64

In [108]:
# Valutiamo la media del BaseRate per ogni paese: ci sono differenze?
baserate_avg = emp_sal_terr.groupby("SalesTerritoryCountry").BaseRate.mean()
baserate_avg

SalesTerritoryCountry
Australia         47.764433
Canada            41.592550
France            41.592550
Germany           47.764433
United Kingdom    47.764433
United States     39.909309
Name: BaseRate, dtype: float64